In [1]:
import sys
sys.path.insert(0, '../../')

In [ ]:
import backtrader as bt
import pandas as pd
import numpy as np
import datetime
from copy import deepcopy
from backtrader.data_reader import get_fund_nav_data, get_all_fund_codes, get_date_range, test_connection

In [16]:
def format_nav_data(df):
    df_formatted = df.rename(columns={'open_price': 'open',
                                      'close_price': 'close',
                                      'high_price': 'high',
                                      'low_price': 'low',
                                      'nav_date': 'datetime'})
    
    df_formatted = df_formatted[['datetime','fund_code', 'open', 'high', 'low', 'close', 'volume']].copy()
    df_formatted['volume'] = df_formatted['volume'].astype(int)
    df_formatted['open'] = df_formatted['open'].astype(float)
    df_formatted['high'] = df_formatted['high'].astype(float)
    df_formatted['low'] = df_formatted['low'].astype(float)
    df_formatted['close'] = df_formatted['close'].astype(float)
    df_formatted['openinterest'] = 0
    df_formatted['datetime'] = pd.to_datetime(df_formatted['datetime'])
    return df_formatted

# 一、读取日度行情表

表内字段就是 Backtrader 默认情况下要求输入的 7 个字段： 'datetime' 、'open'、'high'、'low'、'close'、'volume'、'openinterest'，外加一个 'sec_code' 股票代码字段。

In [13]:
daily_price = get_fund_nav_data(fund_codes=['513310','518880'],
                              start_date='2024-01-01', end_date='2026-01-01')

In [17]:
#daily_price = pd.read_csv("./data/daily_price.csv", parse_dates=['datetime'])
daily_price = format_nav_data(daily_price)
daily_price.head()

,datetime,fund_code,open,high,low,close,volume,openinterest
0,2024-01-02,513310,1.235,1.237,1.224,1.228,130393,0
1,2024-01-03,513310,1.216,1.216,1.193,1.193,152362,0
2,2024-01-04,513310,1.188,1.194,1.177,1.183,105238,0
3,2024-01-05,513310,1.185,1.188,1.165,1.169,248912,0
4,2024-01-08,513310,1.173,1.174,1.147,1.148,108920,0


In [18]:
# 筛选 513310 和 518880 2只基金的数据集
data1 = daily_price.query(f"fund_code=='513310'").set_index('datetime').drop(columns=['fund_code'])
data2 = daily_price.query(f"fund_code=='518880'").set_index('datetime').drop(columns=['fund_code'])
data2

,open,high,low,close,volume,openinterest
datetime,,,,,,
2024-01-02,4.643,4.667,4.643,4.666,2062784,0
2024-01-03,4.658,4.666,4.655,4.663,1111962,0
2024-01-04,4.640,4.656,4.633,4.651,1149663,0
2024-01-05,4.659,4.665,4.656,4.664,1267497,0
2024-01-08,4.663,4.667,4.646,4.650,1161639,0
...,...,...,...,...,...,...
2025-12-25,9.571,9.580,9.539,9.566,4311576,0
2025-12-26,9.670,9.680,9.618,9.650,4898253,0
2025-12-29,9.640,9.667,9.550,9.563,7144967,0


In [21]:
# 导入指标库
import backtrader.indicators as btind # 导入策略分析模块

# 一、 建议在 __init__() 中提前计算指标

In [22]:
class MyStrategy(bt.Strategy):
	# 先在 __init__ 中提前算好指标
    def __init__(self):
        sma1 = btind.SimpleMovingAverage(self.data)
        ema1 = btind.ExponentialMovingAverage()
        close_over_sma = self.data.close > sma1
        close_over_ema = self.data.close > ema1
        sma_ema_diff = sma1 - ema1
        # 生成交易信号
        buy_sig = bt.And(close_over_sma, close_over_ema, sma_ema_diff > 0)
    # 在 next 中直接调用计算好的指标
    def next(self):
        if buy_sig:
            self.buy()

# 二、计算指标时的各种简写形式

In [24]:
class TestStrategy(bt.Strategy):
    def __init__(self):
        # 最简方式：直接省略指向的数据集
        self.sma1 = btind.SimpleMovingAverage(period=10)
        # 只指定第一个数据表格
        self.sma2 = btind.SMA(self.data, period=10)
        # 指定第一个数据表格的close 线
        self.sma3 = btind.SMA(self.data.close, period=10)
        # 完整写法
        self.sma4 = btind.SMA(self.datas[0].lines[0], period=10)
        # 指标函数也支持简写 SimpleMovingAverage → SMA
        
    def next(self):
        # 提取当前时间点
        print('datetime', self.datas[0].datetime.date(0))
        # 打印当日、昨日、前日的均线
        print('sma1',self.sma1.get(ago=0, size=3))
        print('sma2',self.sma2.get(ago=0, size=3))
        print('sma3',self.sma3.get(ago=0, size=3))
        print('sma4',self.sma4.get(ago=0, size=3))
        
cerebro = bt.Cerebro()
st_date = datetime.datetime(2024,1,1)
end_date = datetime.datetime(2026,1,1)
datafeed1 = bt.feeds.PandasData(dataname=data1, fromdate=st_date, todate=end_date)
cerebro.adddata(datafeed1, name='513310')       
datafeed2 = bt.feeds.PandasData(dataname=data2, fromdate=st_date, todate=end_date)
cerebro.adddata(datafeed2, name='518880')  
cerebro.addstrategy(TestStrategy)
rasult = cerebro.run()

datetime 2024-01-15
sma1 array('d', [nan, nan, 1.1608])
sma2 array('d', [nan, nan, 1.1608])
sma3 array('d', [nan, nan, 1.1608])
sma4 array('d', [nan, nan, 1.1608])
datetime 2024-01-16
sma1 array('d', [nan, 1.1608, 1.149])
sma2 array('d', [nan, 1.1608, 1.149])
sma3 array('d', [nan, 1.1608, 1.149])
sma4 array('d', [nan, 1.1608, 1.149])
datetime 2024-01-17
sma1 array('d', [1.1608, 1.149, 1.1381000000000001])
sma2 array('d', [1.1608, 1.149, 1.1381000000000001])
sma3 array('d', [1.1608, 1.149, 1.1381000000000001])
sma4 array('d', [1.1608, 1.149, 1.1381000000000001])
datetime 2024-01-18
sma1 array('d', [1.149, 1.1381000000000001, 1.1318])
sma2 array('d', [1.149, 1.1381000000000001, 1.1318])
sma3 array('d', [1.149, 1.1381000000000001, 1.1318])
sma4 array('d', [1.149, 1.1381000000000001, 1.1318])
datetime 2024-01-19
sma1 array('d', [1.1381000000000001, 1.1318, 1.1293])
sma2 array('d', [1.1381000000000001, 1.1318, 1.1293])
sma3 array('d', [1.1381000000000001, 1.1318, 1.1293])
sma4 array('d', [1

# 三、调用指标时的各种简写形式

In [25]:
class TestStrategy(bt.Strategy):
    
    def __init__(self):
        self.sma5 = btind.SimpleMovingAverage(period=5) # 5日均线
        self.sma10 = btind.SimpleMovingAverage(period=10) # 10日均线
        self.buy_sig = self.sma5 > self.sma10 # 5日均线上穿10日均线
      
    def next(self):
        # 提取当前时间点
        print('datetime', self.datas[0].datetime.date(0))
        # 打印当前值
        print('close', self.data.close[0], self.data.close)
        print('sma5', self.sma5[0], self.sma5)
        print('sma10', self.sma10[0], self.sma10)
        print('buy_sig', self.buy_sig[0], self.buy_sig)
        # 比较收盘价与均线的大小
        if self.data.close[0] > self.sma5:
            print('------收盘价上穿5日均线------')
        if self.data.close[0] > self.sma10:
            print('------收盘价上穿10日均线------')
        if self.buy_sig:
            print('------ buy ------')
        
cerebro2 = bt.Cerebro()
st_date = datetime.datetime(2025,1,1)
end_date = datetime.datetime(2026,1,1)
datafeed1 = bt.feeds.PandasData(dataname=data1, fromdate=st_date, todate=end_date)
cerebro2.adddata(datafeed1, name='600466.SH')       
cerebro2.addstrategy(TestStrategy)
rasult = cerebro2.run()


datetime 2025-01-15
close 1.5 <backtrader.linebuffer.LineBuffer object at 0x000001F7CFA89ED0>
sma5 1.4942 <backtrader.indicators.sma.SimpleMovingAverage object at 0x000001F7CFA2F880>
sma10 1.4724 <backtrader.indicators.sma.SimpleMovingAverage object at 0x000001F7CFA2F100>
buy_sig 1.0 <backtrader.linebuffer.LinesOperation object at 0x000001F7CFA2F160>
------收盘价上穿5日均线------
------收盘价上穿10日均线------
------ buy ------
datetime 2025-01-16
close 1.495 <backtrader.linebuffer.LineBuffer object at 0x000001F7CFA89ED0>
sma5 1.4914 <backtrader.indicators.sma.SimpleMovingAverage object at 0x000001F7CFA2F880>
sma10 1.4823 <backtrader.indicators.sma.SimpleMovingAverage object at 0x000001F7CFA2F100>
buy_sig 1.0 <backtrader.linebuffer.LinesOperation object at 0x000001F7CFA2F160>
------收盘价上穿5日均线------
------收盘价上穿10日均线------
------ buy ------
datetime 2025-01-17
close 1.521 <backtrader.linebuffer.LineBuffer object at 0x000001F7CFA89ED0>
sma5 1.4958 <backtrader.indicators.sma.SimpleMovingAverage object at 0

# 四、好用的运算指标

In [27]:
import backtrader.indicators as btind # 导入策略分析模块
class TestStrategy(bt.Strategy):
    
    def __init__(self):
        self.sma5 = btind.SimpleMovingAverage(period=5) # 5日均线
        self.sma10 = btind.SimpleMovingAverage(period=10) # 10日均线
        # bt.And 中所有条件都满足时返回 1；有一个条件不满足就返回 0, self.data = self.data.close
        self.And = bt.And(self.data.close>self.sma5, self.data.close>self.sma10, self.sma5>self.sma10)
        # bt.Or 中有一个条件满足时就返回 1；所有条件都不满足时返回 0
        self.Or = bt.Or(self.data.close>self.sma5, self.data.close>self.sma10, self.sma5>self.sma10)
        # bt.If(a, b, c) 如果满足条件 a，就返回 b，否则返回 c
        self.If = bt.If(self.data.close>self.sma5,1000, 5000)
        # bt.All,同 bt.And
        self.All = bt.All(self.data.close>self.sma5, self.data.close>self.sma10, self.sma5>self.sma10)
        # bt.Any，同 bt.Or
        self.Any = bt.Any(self.data.close>self.sma5, self.data.close>self.sma10, self.sma5>self.sma10)
        # bt.Max，返回同一时刻所有指标中的最大值
        self.Max = bt.Max(self.data.close, self.sma10, self.sma5)
        # bt.Min，返回同一时刻所有指标中的最小值
        self.Min = bt.Min(self.data, self.sma10, self.sma5)
        # bt.Sum，对同一时刻所有指标进行求和
        self.Sum = bt.Sum(self.data, self.sma10, self.sma5)
        # bt.Cmp(a,b), 如果 a>b ，返回 1；否则返回 -1
        self.Cmp = bt.Cmp(self.data, self.sma5)

        
    def next(self):
        print('---------- datetime',self.data.datetime.date(0), '------------------')
        print('close:', self.data[0], 'ma5:', self.sma5[0], 'ma10:', self.sma10[0])
        print('close>ma5',self.data>self.sma5, 'close>ma10',self.data>self.sma10, 'ma5>ma10', self.sma5>self.sma10)
        print('self.And', self.And[0], self.data>self.sma5 and self.data>self.sma10 and self.sma5>self.sma10)
        print('self.Or', self.Or[0], self.data>self.sma5 or self.data>self.sma10 or self.sma5>self.sma10)
        print('self.If', self.If[0], 1000 if self.data>self.sma5 else 5000)
        print('self.All',self.All[0], self.data>self.sma5 and self.data>self.sma10 and self.sma5>self.sma10)
        print('self.Any', self.Any[0], self.data>self.sma5 or self.data>self.sma10 or self.sma5>self.sma10)
        print('self.Max',self.Max[0], max([self.data[0], self.sma10[0], self.sma5[0]]))
        print('self.Min', self.Min[0], min([self.data[0], self.sma10[0], self.sma5[0]]))
        print('self.Sum', self.Sum[0], sum([self.data[0], self.sma10[0], self.sma5[0]]))
        print('self.Cmp', self.Cmp[0], 1 if self.data>self.sma5 else -1)  
        
cerebro3 = bt.Cerebro()
st_date = datetime.datetime(2025,1,1)
ed_date = datetime.datetime(2025,6,1)
datafeed1 = bt.feeds.PandasData(dataname=data1, fromdate=st_date, todate=ed_date)
cerebro3.adddata(datafeed1, name='600466.SH')

cerebro3.addstrategy(TestStrategy)
rasult = cerebro3.run()

---------- datetime 2025-01-15 ------------------
close: 1.5 ma5: 1.4942 ma10: 1.4724
close>ma5 True close>ma10 True ma5>ma10 True
self.And 1.0 True
self.Or 1.0 True
self.If 1000.0 1000
self.All 1.0 True
self.Any 1.0 True
self.Max 1.5 1.5
self.Min 1.4724 1.4724
self.Sum 4.4666 4.4666
self.Cmp 1.0 1
---------- datetime 2025-01-16 ------------------
close: 1.495 ma5: 1.4914 ma10: 1.4823
close>ma5 True close>ma10 True ma5>ma10 True
self.And 1.0 True
self.Or 1.0 True
self.If 1000.0 1000
self.All 1.0 True
self.Any 1.0 True
self.Max 1.495 1.495
self.Min 1.4823 1.4823
self.Sum 4.4687 4.4687
self.Cmp 1.0 1
---------- datetime 2025-01-17 ------------------
close: 1.521 ma5: 1.4958 ma10: 1.4927
close>ma5 True close>ma10 True ma5>ma10 True
self.And 1.0 True
self.Or 1.0 True
self.If 1000.0 1000
self.All 1.0 True
self.Any 1.0 True
self.Max 1.521 1.521
self.Min 1.4927 1.4927
self.Sum 4.5095 4.5095
self.Cmp 1.0 1
---------- datetime 2025-01-20 ------------------
close: 1.523 ma5: 1.5068 ma10: 1.4998


# 五、如何对齐不同周期的指标

```python
# self.data0 是日度行情、self.data1 是月度行情
self.month = btind.xxx(self.data1) # 计算返回的 self.month 指标也是月度的
# 选择指标对象中的第一条 line 进行对齐
self.sellsignal = self.data0.close < self.month.lines[0]()
# 对齐整个指标对象
self.month_ = self.month() 
self.signal = self.data0.close < self.month_.lines[0] 

cerebro.run(runonce=False)
```

# 六、在 Backtrader 中调用 TA-Lib 库

In [ ]:
class TALibStrategy(bt.Strategy):
    def __init__(self):
        # 计算 5 日均线
        bt.talib.SMA(self.data.close, timeperiod=5)
        bt.indicators.SMA(self.data, period=5)
        # 计算布林带
        bt.talib.BBANDS(self.data, timeperiod=25)
        bt.indicators.BollingerBands(self.data, period=25)

# 七、自定义新指标

In [ ]:
class MyInd(bt.Indicator):
    lines = (xxx,xxx, ) # 最后一个 “,” 别省略
    params = ((xxx, n),) # 最后一个 “,” 别省略
    
    def __init__(self):
        '''可选'''
        pass
    
    def next(self):
        '''可选'''
        pass
    
    def once(self):
        '''可选'''
        pass 
    
    plotinfo = dict(...)
    plotlines = dict(...)
    ...

In [ ]:
class DummyInd(bt.Indicator):
    # 将计算的指标命名为 'dummyline'，后面调用这根 line 的方式有： 
    # self.lines.dummyline ↔ self.l.dummyline ↔ self.dummyline
    lines = ('dummyline',)
    # 定义参数，后面调用这个参数的方式有：
    # self.params.xxx ↔ self.p.xxx
    params = (('value', 5),)
    
    def __init__(self):
        self.l.dummyline = bt.Max(0.0, self.p.value)
    
    def next(self):
        self.l.dummyline[0] = max(0.0, self.p.value)
   
    def once(self, start, end):
        dummy_array = self.l.dummyline.array
        for i in xrange(start, end):
            dummy_array[i] = max(0.0, self.p.value)

In [ ]:
class My_MACD(bt.Indicator):
    lines = ('macd', 'signal', 'histo')
    params = (('period_me1',12), 
              ('period_me2', 26), 
              ('period_signal', 9),)
    def __init__(self):
        me1 = EMA(self.data, period=self.p.period_me1)
        me2 = EMA(self.data, period=self.p.period_me2)
        self.l.macd = me1 - me2
        self.l.signal = EMA(self.l.macd, period=self.p.period_signal)
        self.l.histo = self.l.macd - self.l.signal